<a href="https://colab.research.google.com/github/sangeethapk/AI-ML/blob/main/IntermediateAssesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-processing trining dataset

In [ ]:
#Load the data
import pandas as pd
train_df=pd.read_csv('/content/train_LZdllcl.csv')
train_df.head()

In [ ]:
#drop user id
train_df.drop('employee_id',axis=1,inplace=True)

In [ ]:
train_df.info()

In [ ]:
train_df.boxplot()

In [ ]:
skewness_values = train_df.select_dtypes(include=['number']).skew()
print(skewness_values)

In [ ]:
import pandas as pd

# Copy to avoid changing original
df = train_df.copy()

# List of skewed columns to cap (based on your skewness results)
skewed_columns = [
    'no_of_trainings',
    'length_of_service',
    'awards_won?',
    'age'
]

for col in skewed_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_cap = Q1 - 1.5 * IQR
    upper_cap = Q3 + 1.5 * IQR

    # Apply capping
    df[col] = df[col].clip(lower=lower_cap, upper=upper_cap)

# Re-check skewness after capping
new_skewness = df[skewed_columns].skew()
print("Skewness after IQR capping:")
print(new_skewness)


In [ ]:
import numpy as np

df['length_of_service'] = np.log1p(df['length_of_service'])
df['age'] = np.log1p(df['age'])
print(df[['length_of_service', 'age']].skew())



Split features and target

In [ ]:
#get X and y value
X=df.drop('is_promoted',axis=1)
y=df['is_promoted']
#

In [ ]:
#find imbalance in y
y.value_counts()

In [ ]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Remove NUll value from X_train

In [ ]:
#Handle missing values in X_train
X_train.isnull().sum()

In [ ]:
#impute missing values' most frequent' value for education
mode_education=X_train['education'].mode()[0]
X_train['education']=X_train['education'].fillna(mode_education)
#impute previous_year_rating
X_train['previous_year_rating']=X_train['previous_year_rating'].fillna(X_train['previous_year_rating'].median())


In [ ]:
X_train.isnull().sum()

Remove null value from X_test using smae method

In [ ]:
X_test.isnull().sum()

In [ ]:
#impute missing values' most frequent' value for education
mode_education=X_test['education'].mode()[0]
X_test['education']=X_test['education'].fillna(mode_education)
#impute previous_year_rating
X_test['previous_year_rating']=X_test['previous_year_rating'].fillna(X_test['previous_year_rating'].median())

In [ ]:
X_test.isnull().sum()

Encode

In [ ]:
#encode X_train and X_test data using for loop for all cat values

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in X_train.select_dtypes(include=['object']):
  X_train[col]=le.fit_transform(X_train[col])
  X_test[col]=le.transform(X_test[col])



In [ ]:
X_train

Scale

In [ ]:
#scale X_train and X_test numeric values
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
for col in X_train.select_dtypes(include=['number']):
  X_train[col]=scaler.fit_transform(X_train[[col]])
  X_test[col]=scaler.transform(X_test[[col]])


SMOTE

In [ ]:
#Smote
from imblearn.over_sampling import SMOTE
smote=SMOTE()
X_train,y_train=smote.fit_resample(X_train,y_train)

Train the models

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier



# List of models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'SVM': SVC(),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': LGBMClassifier()
}

# Train and evaluate
results = []

for name, model in models.items():
    # Use scaled data for models that are sensitive to feature scale
    if name in ['SVM', 'Logistic Regression', 'KNN', 'Naive Bayes']:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0),
        'F1 Score': f1_score(y_test, y_pred, zero_division=0)
    })

#. Show results as table
results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False)
print(results_df)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:42:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 40086, number of negative: 40086
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004697 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2062
[LightGBM] [Info] Number of data points in the train set: 80172, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                 Model  Accuracy  Precision    Recall  F1 Score
8             LightGBM  0.942073   0.884507  0.345815  0.497229
7              XGBoost  0.938880   0.790244  0.356828  0.491654
1        Decision Tree  0.899197   0.406812  0.473568  0.437659
2        Random Forest  0.920909   0.534338  0.351322  0.423920
3    Gradient Boosting  0.829502   0.265037  0.596916  0.367084
4                  SVM  0.724138 

Hyperparameter Tunning

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


# Models & Hyperparameter Grids
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [10, 20, None]
        }
    },
    'SVM': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7]
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'params': {}  # No hyperparams to tune for basic NB
    }
}

#  Train & Evaluate
results = []

for name, mp in model_params.items():
    grid = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='f1', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    results.append({
        'Model': name,
        'Best Params': grid.best_params_,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0),
        'F1 Score': f1_score(y_test, y_pred, zero_division=0)
    })

# 4. Display Results
results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False)
pd.set_option('display.max_colwidth', None)
print(results_df)
